In [3]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys, git

import uproot as ut
import dask_awkward as dak 
import awkward as ak


sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
filename = fc.eightb.preselection.t8btag_minmass.NMSSM_XYY_YToHH_8b_MX_1000_MY_450

In [130]:
import concurrent.futures as cf

import logging

logger = logging.getLogger('studies')
logger.setLevel(logging.DEBUG)

class AsyncTree:
    def __init__(self, filename, treename, branches, executor):
        self.filename = filename
        self.treename = treename
        self.fields = branches

        self.async_tree = executor.submit( self.load_async, branches )

    def load_async(self, branches):
        logger.debug(f'Loading branches {branches} from {self.filename}')
        with ut.open(f'{self.filename}:{self.treename}') as tree:
            tree = tree.arrays(branches)
        logger.debug(f'Loaded branches {branches} from {self.filename}')
        return tree

    def result(self):
        return self.async_tree.result()

class UserTree:
    pool = None

    @staticmethod
    def set_pool(num_workers):
        UserTree.pool = cf.ThreadPoolExecutor(max_workers=num_workers)

    def __init__(self, filename, treename='sixBtree', branches=[], num_workers=1):
        self.filename = filename
        self.treename = treename

        self.ut_file = ut.open(filename)
        self.ut_ttree = self.ut_file[treename]

        if any(branches):
            self.ttree = self.ut_ttree.arrays(branches)
        else:
            self.ttree = None    
        self.fields = [ branch.name for branch in self.ut_ttree.branches ]

        self.async_ttree = None


    def _load_branches(self, branches):
        if not isinstance(branches, list):
            branches = [branches]

        if UserTree.pool:
            self.async_tree = AsyncTree(self.filename, self.treename, branches, UserTree.pool)
            return

        branches = self.ut_ttree.arrays(branches)
        return branches

    def load_branches(self, branches):
        branches = self._load_branches(branches)

        if branches is None:
            return

        if self.ttree is None:
            self.ttree = branches
        else:
            for field, branch in zip( branches.fields, ak.unzip(branches) ):
                self.ttree[field] = branch

    def load_async(self):
        branches = self.async_tree.result()
        logger.debug(f'Fetching async branches {repr(branches)} from {self.filename}')

        if self.ttree is None:
            self.ttree = branches
        else:
            for field, branch in zip( branches.fields, ak.unzip(branches) ):
                self.ttree[field] = branch
        self.async_tree = None

    def __repr__(self):
        string = repr(self.ut_ttree)
        if not self.ttree is None:
            string += '\n   ' + repr(self.ttree)
        return string
    
    def __getitem__(self, key):
        if (self.ttree is None) or (key not in self.ttree.fields):
            self.load_branches(key)

        if self.async_tree and key in self.async_tree.fields:
            self.load_async()
        
        return self.ttree[key]
    def __getattr__(self, key):
        return self[key]

        



In [131]:
UserTree.set_pool(4)

In [132]:
trees = [ UserTree(fc.fs.default.fullpath(f)) for f in fc.eightb.preselection.t8btag_minmass.signal_list ]


In [133]:
for tree in trees:
    tree.load_branches('X_pt')

DEBUG:studies:Loading branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_300/ntuple.root
DEBUG:studies:Loading branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1000_MY_450/ntuple.root
DEBUG:studies:Loading branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1200_MY_500/ntuple.root


In [134]:
tree.X_pt

DEBUG:studies:Loaded branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_300/ntuple.root
DEBUG:studies:Loaded branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1000_MY_450/ntuple.root
DEBUG:studies:Loaded branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1200_MY_500/ntuple.root
DEBUG:studies:Loading branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1200_MY_500/ntuple.root
DEBUG:studies:Loaded branches ['X_pt'] from root://cmseos.fnal.gov//store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1200

<Array [83.9, 200, 136, ... 16.9, 78.4, 136] type='676524 * float32'>

In [98]:
tree.X_pt

<Array [117, 41.7, 55.2, ... 51.1, 53, 304] type='618416 * float32'>

In [99]:
tree

<TTree 'sixBtree' (399 branches) at 0x7f4b8d203d30>
   <Array [{X_pt: 117}, ... {X_pt: 304}] type='618416 * {"X_pt": float32}'>

In [100]:
tree.X_m

<Array [931, 1e+03, ... 827, 1.03e+03] type='618416 * float32'>

In [102]:
tree

<TTree 'sixBtree' (399 branches) at 0x7f4b8d203d30>
   <Array [{X_pt: 117, ... X_m: 1.03e+03}] type='618416 * {"X_pt": float32, "X_m": ...'>